In [1]:
import awkward as ak
import cupy as cp
import cudf
import numpy as np
import awkward_pandas.cudf
import subprocess

def gpu_mem():
    print(subprocess.check_output("nvidia-smi | grep py", shell=True).split()[-2].decode())

ak.__version__, awkward_pandas.__version__

('2.6.3', '2023.8.1.dev25+g5dd12c5.d20240417')

In [2]:
df = cudf.read_parquet("/floppy/code/awkward/s.parquet")
gpu_mem()

188MiB


In [3]:
df.dtypes

a    list
dtype: object

In [4]:
df.iloc[0]  # each element is list-of-lists

a    [[1, 2, 3], [], [4, 5]]
Name: 0, dtype: list

In [5]:
# series accessor
df.a.ak

In [6]:
# allows all ak.* namespace, many identical to numpy equivalents
dir(df.a.ak)

['all',
 'almost_equal',
 'angle',
 'annotations',
 'any',
 'apply',
 'argcartesian',
 'argcombinations',
 'argmax',
 'argmin',
 'argsort',
 'array',
 'awkward',
 'backend',
 'behavior',
 'behaviors',
 'broadcast_arrays',
 'broadcast_fields',
 'builder',
 'cartesian',
 'categories',
 'combinations',
 'concatenate',
 'contents',
 'copy',
 'corr',
 'count',
 'count_nonzero',
 'covar',
 'cppyy',
 'drop_none',
 'dt',
 'enforce_type',
 'errors',
 'explode',
 'fields',
 'fill_none',
 'firsts',
 'flatten',
 'forms',
 'forth',
 'from_arrow',
 'from_arrow_schema',
 'from_avro_file',
 'from_buffers',
 'from_categorical',
 'from_cupy',
 'from_dlpack',
 'from_feather',
 'from_iter',
 'from_jax',
 'from_json',
 'from_numpy',
 'from_parquet',
 'from_rdataframe',
 'from_regular',
 'full_like',
 'highlevel',
 'imag',
 'index',
 'is_categorical',
 'is_none',
 'is_tuple',
 'is_valid',
 'isclose',
 'jax',
 'linear_fit',
 'local_index',
 'mask',
 'max',
 'mean',
 'merge_option_of_records',
 'merge_union_o

In [7]:
df.a.ak.sum(axis=None)

array(28000000)

In [8]:
# if output was array-like, it stays on the GPU
type(_)

cupy.ndarray

In [9]:
# fast reduction across three levels of nesting
%timeit df.a.ak.sum(axis=None)

6.91 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
# ufunc maintains structure
np.negative(df.a.ak)

<Array [[[-1, -2, -3], ..., [-4, ...]], ...] type='2000000 * var * option[v...'>

In [11]:
gpu_mem()  # created new arrays on GPU, made new cuDF series

362MiB


In [12]:
# operator overload
print((df.a.ak + 1).head())

0    [[2, 3, 4], [], [5, 6]]
1    [[2, 3, 4], [], [5, 6]]
2    [[2, 3, 4], [], [5, 6]]
3    [[2, 3, 4], [], [5, 6]]
4    [[2, 3, 4], [], [5, 6]]
dtype: list


#### numba

In [8]:
import numba.cuda
ak.numba.register_and_check()

@numba.cuda.jit(extensions=[ak.numba.cuda])
def inner_sum(array, out):
    tid = numba.cuda.grid(1)
    if tid < len(array):
        out[tid] = 0
        for i, x in enumerate(array[tid]):
            out[tid] += x

out = cp.empty(len(df.a), dtype="int32")
blocksize = 256
numblocks = (len(df.a) + blocksize - 1) // blocksize
df.a.ak.apply(inner_sum[numblocks, blocksize], out)
out


TypingError: Failed in cuda mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function iadd>) found for signature:
 
 >>> iadd(int32, OptionalType(ak.ArrayView(ak.UnmaskedArrayType(ak.NumpyArrayType(array(int64, 1d, C), {}), {}), None, ())))
 
There are 22 candidate implementations:
  - Of which 20 did not match due to:
  Overload of function 'iadd': File: <numerous>: Line N/A.
    With argument(s): '(int32, OptionalType(ak.ArrayView(ak.UnmaskedArrayType(ak.NumpyArrayType(array(int64, 1d, C), {}), {}), None, ())))':
   No match.
  - Of which 2 did not match due to:
  Operator Overload in function 'iadd': File: unknown: Line unknown.
    With argument(s): '(int32, OptionalType(ak.ArrayView(ak.UnmaskedArrayType(ak.NumpyArrayType(array(int64, 1d, C), {}), {}), None, ())))':
   No match for registered cases:
    * (int64, int64) -> int64
    * (int64, uint64) -> int64
    * (uint64, int64) -> int64
    * (uint64, uint64) -> uint64
    * (float32, floa

In [9]:
df.a.ak.array

<Array [[[1, 2, 3], [], [4, 5]], ..., [[...]]] type='2000000 * var * option...'>

In [11]:
type(inner_sum[numblocks, blocksize])

numba.cuda.dispatcher._LaunchConfiguration

In [12]:
type(inner_sum)

numba.cuda.dispatcher.CUDADispatcher